In [45]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [57]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
# 혹은 options.add_argument("--disable-gpu")

driver = webdriver.chrome.webdriver.WebDriver(executable_path='../chromedriver', chrome_options=options)
driver.get("https://korean.visitseoul.net/hallyu?srchType=&srchOptnCode=&srchCtgry=32&sortOrder=&srchWord=")
wait = WebDriverWait(driver, 10)
wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'ul')))     
# CSS_SELECTOR의 h1이 페이지에 표시될 때 까지 wait에서 설정한 시간만큼 기다리기


/home/kimdooh/.pyenv/versions/kopa-tour-env/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


<selenium.webdriver.remote.webelement.WebElement (session="b88c0a730f4a6c561c94fdd5e1fd49fd", element="3ffa9e39-0bfe-440a-afa8-a59e2684ca58")>

In [72]:
search = driver.find_elements_by_xpath('//*[@id="postSearchFrm"]/section[1]/div[3]/ul/li[3]/a')


In [73]:
search

[<selenium.webdriver.remote.webelement.WebElement (session="b88c0a730f4a6c561c94fdd5e1fd49fd", element="a176c5ea-90a5-4a40-9876-c1883b2ba794")>]

In [74]:
kpop_url_list = dict()

In [165]:
def crawler_url(url)
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    # 혹은 options.add_argument("--disable-gpu")

    driver = webdriver.chrome.webdriver.WebDriver(executable_path='../chromedriver', chrome_options=options)
    url = "https://korean.visitseoul.net/hallyu/?curPage=10"
    driver.get(url)


    try:
        article_list = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'article-list-slide'))
        )
        articles = article_list.find_elements_by_tag_name("li")
        for article in articles:
            a_tag = article.find_element_by_tag_name("a")

            title = a_tag.get_attribute('title')
            href  = a_tag.get_attribute('href')
            href = urllib.parse.unquote(href)
            thumb = a_tag.find_element_by_class_name('thumb').get_attribute('style')

            kpop_url_list[title] = {
                'href': href,
                'thumb' : 'https://korean.visitseoul.net/' +thumb.replace('background-image: url("','').replace('");',''),
            }
            print(a_tag.get_attribute('title'))
            print(a_tag.get_attribute('href'))
            print(a_tag.find_element_by_class_name('thumb').get_attribute('style'))

    except Exception as ex: # 에러 종류
        print('에러가 발생 했습니다', ex) #
        driver.quit()

    driver.quit()
    return kpop_url_list

/home/kimdooh/.pyenv/versions/kopa-tour-env/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


반려견과 함께하는 서울 명소
https://korean.visitseoul.net/hallyu/%ED%95%A8%EA%BB%98-%EC%82%AC%EB%8A%94-%EB%8F%84%EC%8B%9C-%EC%84%9C%EC%9A%B8_/28765
background-image: url("/comm/getImage?srvcId=POST&parentSn=28765&fileTy=POSTTHUMB&fileNo=3&thumbTy=M");
서울의 무인서비스
https://korean.visitseoul.net/hallyu/%EB%AC%B4%EC%9D%B8-in-%EC%84%9C%EC%9A%B8_/28757
background-image: url("/comm/getImage?srvcId=POST&parentSn=28757&fileTy=POSTTHUMB&fileNo=2&thumbTy=M");
옛가게길: 한국과 세계가 함께 숨 쉬는 곳
https://korean.visitseoul.net/hallyu/%EC%98%9B%EA%B0%80%EA%B2%8C%EA%B8%B8-%ED%95%9C%EA%B5%AD%EA%B3%BC-%EC%84%B8%EA%B3%84%EA%B0%80-%ED%95%A8%EA%BB%98-%EC%88%A8-%EC%89%AC%EB%8A%94-%EA%B3%B3_/28661
background-image: url("/comm/getImage?srvcId=POST&parentSn=28661&fileTy=POSTTHUMB&fileNo=3&thumbTy=M");
알아두면 쓸모 있는 서울 이색박물관
https://korean.visitseoul.net/hallyu/%EC%95%8C%EC%95%84%EB%91%90%EB%A9%B4-%EC%93%B8%EB%AA%A8-%EC%9E%88%EB%8A%94-%EC%84%9C%EC%9A%B8-%EC%9D%B4%EC%83%89%EB%B0%95%EB%AC%BC%EA%B4%80_/28607
background-image: url("/comm/getIm

In [170]:
import os 
import json 

kpop_all = kpop_url_list
for i in kpop_all:
    url = kpop_all[i]['href']
    url = urllib.parse.unquote(url)
    kpop_all[i]['href'] = url

json_data = json.dumps(kpop_url_list)

In [201]:
with open('./kpop_url_range_1_10.json', 'w', encoding='UTF-8') as outfile:
    json.dump(kpop_all, outfile, ensure_ascii=False)

In [157]:
with open('./kpop_url_range_1_10.json', 'r') as file:
    json_data = json.load(file)

In [205]:
for title in kpop_url_list:
    kpop_url_list[title] = 

{'당신도 봉준호 마니아라면 봉하이브 투어': {'href': 'https://korean.visitseoul.net/hallyu/봉하이브-투어_/34310',
  'thumb': 'https://korean.visitseoul.net//comm/getImage?srvcId=POST&parentSn=34310&fileTy=POSTTHUMB&fileNo=1&thumbTy=M'},
 '<필이 좋은 여행> 필이 찾은 맛과 이야기 있는 서울': {'href': 'https://korean.visitseoul.net/hallyu/필이-좋은-여행-필이-찾은-맛과-이야기-있는-서울_/34309',
  'thumb': 'https://korean.visitseoul.net//comm/getImage?srvcId=POST&parentSn=34309&fileTy=POSTTHUMB&fileNo=1&thumbTy=M'},
 '꺼지지 않는 열풍! <사랑의 불시착> 속 촬영지': {'href': 'https://korean.visitseoul.net/hallyu/꺼지지-않는-열풍-사랑의-불시착-속-촬영지_/34308',
  'thumb': 'https://korean.visitseoul.net//comm/getImage?srvcId=POST&parentSn=34308&fileTy=POSTTHUMB&fileNo=1&thumbTy=M'},
 '근대역사 골목산책 「우리슈퍼」 코스': {'href': 'https://korean.visitseoul.net/hallyu/근대역사-골목산책-우리슈퍼-코스_/34204',
  'thumb': 'https://korean.visitseoul.net//comm/getImage?srvcId=POST&parentSn=34204&fileTy=POSTTHUMB&fileNo=1&thumbTy=M'},
 '서울라이트 일상 체험 「피자시대 코스」 ': {'href': 'https://korean.visitseoul.net/hallyu/서울라이트-일상-체험-피자시대-

In [202]:
def crawler_url(url)
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    # 혹은 options.add_argument("--disable-gpu")

    driver = webdriver.chrome.webdriver.WebDriver(executable_path='../chromedriver', chrome_options=options)
    driver.get(url)


    try:
        article_list = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'article-list-slide'))
        )
        articles = article_list.find_elements_by_tag_name("li")
        for article in articles:
            a_tag = article.find_element_by_tag_name("a")

            title = a_tag.get_attribute('title')
            href  = a_tag.get_attribute('href')
            href = urllib.parse.unquote(href)
            thumb = a_tag.find_element_by_class_name('thumb').get_attribute('style')

            kpop_url_list[title] = {
                'href': href,
                'thumb' : 'https://korean.visitseoul.net/' +thumb.replace('background-image: url("','').replace('");',''),
            }
            print(a_tag.get_attribute('title'))
            print(a_tag.get_attribute('href'))
            print(a_tag.find_element_by_class_name('thumb').get_attribute('style'))

    except Exception as ex: # 에러 종류
        print('에러가 발생 했습니다', ex) #
        driver.quit()

    driver.quit()
    return kpop_url_list